In [ ]:
s_sixco_fg10 = load_nc(data_path(src_dir, 'Gridmet_WIND/wind_gusts_2014.nc'))

print(ds_sixco_fg10.variables.keys())

In [ ]:
# Get the array of latitudes, longitudes, and time
lat = ds_sixco_fg10.variables['latitude'][:]
lon = ds_sixco_fg10.variables['longitude'][:]
time = ds_sixco_fg10.variables['time'][:]

# Calculate the pixel size
dx = lon[1] - lon[0]
dy = lat[0] - lat[1]  # latitudes may be in reverse order, depending on your data

# Define the transformation
transform = from_origin(lon[0] - dx / 2, lat[0] + dy / 2, dx, dy)

# Define the dimensions
height = lat.shape[0]
width = lon.shape[0]
count = time.shape[0]

# Create the raster file
with rasterio.open(
        'Output/fg10.tif',
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=count,
        dtype='float32',  # assuming your data is float; change if otherwise
        crs='+proj=latlong',  # again, assuming a simple lat/long projection; change if otherwise
        transform=transform
) as dst:
    for t in range(count):
        dst.write(ds_sixco_fg10.variables['fg10'][t].filled(np.nan), t + 1)


In [ ]:
# Open the NetCDF file
ds_sixco_fg10 = xr.open_dataset('Gridmet_WIND/wind_gusts_2014.nc')

# Note that xarray automatically plots a histogram if the data is 1D,
# or a 2D image if the data is 2D. Here, let's plot the 'fg10' variable.
ds_sixco_fg10['fg10'].plot()

In [ ]:
ds_sixco_fg10 = xr.open_dataset('Gridmet_WIND/wind_gusts_2014.nc')
plot = ds_sixco_fg10['fg10'].hvplot('longitude', 'latitude')
plot


In [ ]:
# Read the NetCDF file
ds_sixco_fg10 = xr.open_dataset('Gridmet_WIND/wind_gusts_2014.nc')

# Convert the xarray Dataset to a DataFrame
df = ds_sixco_fg10.to_dataframe().reset_index()

# Use Seaborn to create plots (for example, a scatterplot)
sns.scatterplot(x='longitude', y='latitude', hue='fg10', data=df)

# Show the plot
plt.show()

In [ ]:
# Open the NetCDF file
ds_sixco_fg10 = xr.open_dataset('Gridmet_WIND/wind_gusts_2014.nc')

# Let's say we want the data at the first time point:
ds_sixco_fg10_time_slice = ds_sixco_fg10.isel(time=0)  # change '0' to any valid time index

# Convert the xarray Dataset to a DataFrame
df = ds_sixco_fg10_time_slice.to_dataframe().reset_index()

# Pivot the DataFrame to get 2D arrays for lat, lon and your variable
df_pivot = df.pivot(index='latitude', columns='longitude', values='fg10')

# Plot heatmap using seaborn 
plt.figure(figsize=(10, 6))  # Increase size of plot
sns.heatmap(df_pivot)

# Show the plot
plt.show()
